In [ ]:
import torch
import numpy as np
from torchid import metrics
import scipy.io as sio
import matplotlib.pyplot as plt

In [ ]:
noise_std = 0.

In [ ]:
# Configure compute
torch.set_num_threads(10)
out_file = f"batch_y_sim_full_rep_{noise_std:.1f}.pt"

In [ ]:
seq_len_ctx = 400
seq_len_new = 100
batch_size = 32
seq_len = seq_len_ctx + seq_len_new

wh_data = sio.loadmat("wh_data.mat")
batch_u = torch.tensor(wh_data["batch_u"][:batch_size])
batch_y = torch.tensor(wh_data["batch_y"][:batch_size])
batch_y_ctx = batch_y[:, :seq_len_ctx, :]
batch_u_ctx = batch_u[:, :seq_len_ctx, :]
batch_y_new = batch_y[:, seq_len_ctx:, :]
batch_u_new = batch_u[:, seq_len_ctx:, :]
batch_size = batch_u.shape[0]

In [ ]:
batch_y_sim_full_rep = torch.load(out_file)

err_rep = batch_y_sim_full_rep - batch_y # error for all training repetitions R, B, T, C
err_rep_train = err_rep[:, :seq_len_ctx, :, :] # training error for all repetitions R, B, T, C
loss_rep = torch.mean(err_rep_train**2, dim=(2, 3)) # loss for all repetitions R, B, L
idx_best = torch.argmin(torch.nan_to_num(loss_rep, torch.inf),  dim=0) # idx of best training repetition per batch B
batch_y_sim_full_best = batch_y_sim_full_rep[idx_best, torch.arange(batch_size)] # best training repetition full sequence B, T, C

In [ ]:
#batch_y_sim_new = batch_y_sim_full_rep[0, :, cfg.seq_len_ctx:, :] # best training repetition test sequence B, T, C
batch_y_sim_new = batch_y_sim_full_best[:, seq_len_ctx:, :] # best training repetition test sequence B, T, C

In [ ]:
batch_idx = 2
rep_idx = 0

plt.plot(batch_y[batch_idx], 'k', label="y") # identification data
#plt.plot(batch_y_sim_full_rep[:, batch_idx, :, :].squeeze(-1).T, 'b')#, label="yall") # all models
#plt.plot(batch_y_sim_full_rep[rep_idx, batch_idx], 'r') # one model
plt.plot(batch_y_sim_full_best[batch_idx], 'm', label="ybest") # best model
#plt.plot(batch_y[batch_idx], 'k') # repetition of data
plt.legend()
plt.grid()
plt.axvline(seq_len_ctx, color='red')

In [ ]:
batch_sim_err = batch_y_new - batch_y_sim_new
t = np.arange(1, batch_u_new.shape[1]+1) + seq_len_ctx
plt.figure(figsize=(5, 2.5))
#plt.ylim([-3, 3])
plt.plot(t, np.array(batch_y_new.squeeze(-1).T), "k", alpha=0.5);
plt.plot(t, np.nan*np.zeros(t.shape), "k", alpha=1.0, label="$y$")
plt.plot(t, np.array(batch_sim_err.squeeze(-1).T), "r", alpha=0.2);
plt.plot(t, np.nan*np.zeros(t.shape), "r", alpha=1.0, label="$y - \hat y$");
plt.grid(True)
plt.xlabel("time step (-)")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()
#plt.savefig(fig_path / "wh_sim_batch_ml.pdf")

In [ ]:
##%%
skip = 0
rmse_ml = metrics.rmse(batch_y_new.numpy(), batch_y_sim_new.numpy(), time_axis=1)

In [ ]:
np.isnan(rmse_ml).sum()

In [ ]:
rmse_ml = np.nan_to_num(rmse_ml, copy=True, nan=np.nanmean(rmse_ml))

In [ ]:
plt.hist(rmse_ml);

In [ ]:
print(f'rmse: {rmse_ml.mean()}')

In [ ]:
#rmse_ml[rmse_ml < 0.05]